In [1]:
import pandas as pd
import numpy as np

In [8]:
words = set(np.genfromtxt("google-10000-english-usa.txt", dtype=str, delimiter="\n"))

words_list = sorted(words)

emails = pd.read_csv('emails.csv')

lables = emails['spam']

emails = emails.drop('spam', axis=1)

words

{'relax',
 'photographers',
 'palestinian',
 'testimonials',
 'shadow',
 'crap',
 'secretary',
 'disciplines',
 'palmer',
 'extract',
 'taylor',
 'appointed',
 'costs',
 'rental',
 'pig',
 'normal',
 'legally',
 'inspector',
 'lo',
 'conduct',
 'hosting',
 'personnel',
 'ff',
 'ideas',
 'l',
 'comfort',
 'barry',
 'vc',
 'edit',
 'aruba',
 'prefers',
 'evening',
 'sponsored',
 'involved',
 'introduce',
 'clouds',
 'vip',
 'my',
 'dental',
 'overnight',
 'tub',
 'garlic',
 'aquarium',
 'kenny',
 'ing',
 'generally',
 'operated',
 'interact',
 'showtimes',
 'wine',
 'angry',
 'environmental',
 'hello',
 'toshiba',
 'demonstrate',
 'entrepreneurs',
 'experts',
 'scotland',
 'perl',
 'cleaners',
 'proudly',
 'belts',
 'hot',
 'nissan',
 'enterprise',
 'actor',
 'pattern',
 'replica',
 'mother',
 'wiring',
 'farming',
 'firms',
 'consisting',
 'hospitals',
 'access',
 'eve',
 'aquatic',
 'cold',
 'households',
 'nascar',
 'foul',
 'wav',
 'decades',
 'eg',
 'cycles',
 'teams',
 'somalia',
 

In [3]:
emails

,text
0,Subject: naturally irresistible your corporate...
1,Subject: the stock trading gunslinger fanny i...
2,Subject: unbelievable new homes made easy im ...
3,Subject: 4 color printing special request add...
4,"Subject: do not have money , get software cds ..."
...,...
5723,Subject: re : research and development charges...
5724,"Subject: re : receipts from visit jim , than..."
5725,Subject: re : enron case study update wow ! a...
5726,"Subject: re : interest david , please , call..."


In [9]:
"""
making a numpy array of sets. each set in the numpy array contains the unique words of the email corrosponding to the set
"""

email_words = np.array([{'a'}], dtype=object)

for email in emails['text']:
        email_words = np.append(email_words, np.array([set(email.lower().split())], dtype=object), axis=0)


email_words = np.delete(email_words, 0, axis=0)



In [24]:
"""" 
getting the features vector ready. 

each row of the features vector will have 0s and 1s depending on the corrosponding email's word set and the top 10000 words list. 
it is a scipy sparce matricey and should be properly indexed. 



ISSUE: only 1 row is being added to the feature vector.
"""
from scipy.sparse import csr_matrix, vstack

email_words = set([])
intersection_set = set([])

feature_vector = csr_matrix((0, len(words_list)), dtype=bool)

for email in emails["text"]:
    email_words = set(email.lower().split())

    intersection_set = email_words & words
    
    row = []

    for word in words_list:
        if word in intersection_set:
            row.append(True)
        else:
            row.append(False)

    
    row = csr_matrix([row], dtype=bool)
    feature_vector = vstack([feature_vector, row])

feature_vector = csr_matrix(feature_vector, dtype=bool)

feature_vector.toarray



    

<bound method _cs_matrix.toarray of <Compressed Sparse Row sparse matrix of dtype 'bool'
	with 585880 stored elements and shape (5728, 9989)>>

In [27]:
ham_mail = emails[lables == 0]
spam_mail = emails[lables == 1]


In [ ]:
phi_y = spam_mail.shape[0] / (spam_mail.shape[0] + ham_mail[0])

parameters = 